In [1]:
import nltk
import json
import math
from nltk.util import pr
import numpy as np  
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from pattern3.en import singularize
import re
Size = 26709


In [2]:
#Classification problem , either sarcastic or not sarcastic ,2 class problem
alpha = ''
Raw_Data = dict()
X_headline =[]
Y_Sarcastic=[]
i=0


In [3]:
stop_word = []
with open("Stopword-List.txt",'r') as stop:
    for line in stop:
        temp = line.strip()
        stop_word.append(temp)

In [4]:
# #Read data in X and Y labels ,headline and sarcacism score.
# #and tokenize
# #Tokenize
# Tokens = []
# count = 0
# for lines in open('Sarcasm_Headlines_Dataset.json','r'):
#     Raw_Data[i]=json.loads(lines)
#     X_headline.append(Raw_Data[i]["headline"])
#     Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])
    
#     #tokenize here
#     for word in X_headline[i]:
#         if(word == ' '):
#             count += 1
#             Tokens.append(alpha)
#             alpha = ''
#         else:
#             alpha = alpha + word
    
#     i=i+1
def cleaner(uncleaned):
    porter = PorterStemmer()
    lemma = WordNetLemmatizer()
    uncleaned = uncleaned.lower()
    uncleaned = uncleaned.strip()
    uncleaned = uncleaned.translate({ord(i): None for i in '!\\@#-_:$%^&*();.,?/1”2’3“4‘567890\'\"'})
    for i in uncleaned:
        t = ord(i)
        if t < 97 or t>122:
            uncleaned = uncleaned.replace(i, "")
    uncleaned = porter.stem(uncleaned)
    uncleaned = lemma.lemmatize(uncleaned, pos='v')
    # uncleaned = singularize(uncleaned)
    # uncleaned = re.sub(r'ly$', r'', uncleaned)
    # uncleaned = re.sub(r'ed$', r'', uncleaned)
    # uncleaned = re.sub(r'ing$', r'', uncleaned)
    # uncleaned = re.sub(r'nes$', r'', uncleaned)
    return uncleaned



In [5]:
#Read data in X and Y labels ,headline and sarcacism score.
#and tokenize
#Tokenize
from nltk.tokenize import word_tokenize
Tokens = []
count = 0
lexicon = {}
for lines in open('Sarcasm_Headlines_Dataset.json','r'):
    Raw_Data[i]=json.loads(lines)
    X_headline.append(Raw_Data[i]["headline"])
    Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])
    
    #tokenize here
    words = word_tokenize(X_headline[i])
    curr = str(i)
    for word in words:
        if len(word) > 2:
            if word in stop_word:
                continue
            word = cleaner(word)
            Tokens.append(word)

            if word in lexicon:
                if curr in lexicon[word]:
                    lexicon[word][i] += 1
                else:
                    lexicon[word][i] = 1
            else:
                lexicon[word] = {}
                if curr in lexicon[word]:
                    lexicon[word][i] += 1
                else:
                    lexicon[word][i] = 1
    i=i+1
    # print(word)
    # for word in X_headline[i]:
    #     if(word == ' '):
    #         count += 1
    #         Tokens.append(alpha)
    #         alpha = ''
    #     else:
    #         alpha = alpha + word
    
    

    



In [6]:
#Tokenize
# Tokens = []
# count = 0
# for y in range(len(X_headline)):
#     for word in X_headline[y]:
#         if(word == ' '):
#             count += 1
#             Tokens.append(alpha)
#             alpha = ''
#         else:
#             alpha = alpha + word
# print(Tokens)
print("The word count is: "+str(len(Tokens)))
print("Number of docs are: " + str(i))
print(lexicon)
print("dictionary size is",len(lexicon))
# print(type(Raw_Data))

The word count is: 211132
Number of docs are: 26709
{'former': {0: 1, 226: 1, 488: 1, 2681: 1, 2800: 1, 2935: 1, 3200: 1, 3280: 1, 3511: 1, 3547: 1, 3647: 1, 3960: 1, 3987: 1, 4004: 1, 4607: 1, 4789: 1, 4929: 1, 5573: 1, 6235: 1, 6658: 1, 7066: 1, 7293: 1, 7869: 1, 8402: 1, 8428: 1, 8803: 1, 8996: 1, 9409: 1, 9672: 1, 9726: 1, 9824: 1, 9877: 1, 9904: 1, 10580: 1, 10605: 1, 11002: 1, 11140: 1, 11142: 1, 11609: 1, 12352: 1, 12949: 1, 13469: 1, 13926: 1, 14358: 1, 14703: 1, 14969: 1, 15133: 1, 15271: 1, 15320: 1, 15335: 1, 15412: 1, 15763: 1, 15792: 1, 16337: 1, 16413: 1, 16483: 1, 16539: 1, 16650: 1, 16744: 1, 17005: 1, 17197: 1, 17488: 1, 17646: 1, 17779: 1, 17812: 1, 18058: 1, 18242: 1, 18286: 1, 18570: 1, 18756: 1, 18990: 1, 19153: 1, 19338: 1, 19621: 1, 19648: 1, 20066: 1, 20231: 1, 20398: 1, 20587: 1, 20812: 1, 21112: 1, 21401: 1, 21599: 1, 21739: 1, 21980: 1, 22167: 1, 22169: 1, 22270: 1, 22669: 1, 23187: 1, 23195: 1, 23200: 1, 23417: 1, 23893: 1, 23924: 1, 24045: 1, 24225: 1, 2439

In [7]:
def make_idf(lexicon):
    idf = dict()
    N = Size
    for term,docs in lexicon.items():
        df = len(docs)
        temp = math.log(N / df)
        idf[term] = temp
    return idf

In [8]:
def init_tf_idf_lexicon(lexicon):
    tf_idf = dict()
    for keys,values in lexicon.items():
        tf_idf[keys] = {}
        for leg in range(0,Size+1):
            tf_idf[keys][leg] = float(0)
    return tf_idf

def tf_idf_lexicon(lexicon,idf):
    tf_idf = init_tf_idf_lexicon(lexicon)
    for term,docs in lexicon.items():
        for docNo,tf in docs.items():
            tf_idf[term][docNo] = tf * idf[term]

    return tf_idf

In [9]:
idf = make_idf(lexicon)
# print(idf)

# print(tf_idf)

In [10]:
tf_idf = tf_idf_lexicon(lexicon,idf)

MemoryError: 